In [16]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

     |████████████████████████████████| 12.0 MB 16.1 MB/s 


In [17]:
#Packages

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import spacy
import nltk
nltk.download('wordnet')
nltk.download('punkt')
!pip install pattern 
from pattern.en import conjugate
import spacy.cli
spacy.cli.download("en_core_web_lg")
from spacy.tokens import Doc
import re
from pattern.en import singularize
import textwrap

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [18]:
nlp = spacy.load("en_core_web_lg")

In [19]:
#We build a fonction that indicates if a word is singular of plural 
def isplural(pluralForm):
     singularForm = singularize(pluralForm)
     plural = True if pluralForm is not singularForm else False
     return plural

isplural("parents")



True

In [20]:
#Define a wrapper that we allow the text to fit well the screen
wrapper = textwrap.TextWrapper(width=120)



In [26]:
def chatbot():

  ###### Initialisation of the conversation ######

  print("- Socrate: Hello I am Socrate!, I am here to help you, what is your problem at the moment?")

  answer1=input("- User: ")

  ###### Stage 1: Finding the reasons of the problems ######

  doc = nlp(answer1)

  #We replace first person pronouns by second person pronouns
  answer1=re.sub(r'\bi\b', 'you', answer1)
  answer1=re.sub(r'\bI\b', 'you', answer1)
  answer1=re.sub(r'\bmy\b', 'your', answer1)
  answer1=re.sub(r'\bMy\b', 'your', answer1)
  answer1=re.sub(r'\bmine\b', 'yours', answer1)
  answer1=re.sub(r'\bMine\b', 'yours', answer1)
  answer1=re.sub(r'\bam\b', 'are', answer1)
  answer1=re.sub(r'\bme\b', 'you', answer1) 
  answer1=re.sub(r'\bwas\b', 'were', answer1) 

  print(wrapper.fill("- Socrate: Could you think of any reason why "+answer1+"?"))

  ##### The reason is valid only if the sentence contains a verb #####
  verb = []
  while len(verb)==0:
  
    answer2=input("- User: ").lower()
    #We create a copy to keep the original form that will be useful after 
    answer2_copy=answer2
    doc=nlp(answer2)
    
    verb_idx=[]
    for token in doc:
        #We consider verb that is not in "ing" form or in past principle form 
        if token.pos_ in ('VERB', 'CONJ', 'AUX') and token.tag_ !="VBG" and token.tag_ !="VBN"  :
          verb.append(token)
          verb_idx.append(token.i)
    if len(verb)==0:
      print("- Socrate : Sorry I do not understand your answer, your sentence does not contain any verb")
      print(wrapper.fill("- Socrate : Try answering again: could you think of any reason why "+answer1+"?"))


  vb_conj=[]
  for i in range(0,len(verb)):
    #If the world just before the verb (we assume that is the subject) is "I" or a plural subject, we conjugate at the 2nd person
    if str(doc[verb_idx[i]-1]).lower()=="i" or isplural(str(doc[verb_idx[i]-1])):
      vb_conj.append(conjugate(str(verb[i]), tense = "present",person = 2, number = "singular", mood = "indicative", aspect = "imperfective",negated = False))
    #Otherwise we conjugate at the 3rd person
    else:
      vb_conj.append(conjugate(str(verb[i]), tense = "present",person = 3, number = "singular", mood = "indicative", aspect = "imperfective",negated = False))

  for i in range(0,len(verb)):
    answer2=re.sub(str(doc[verb_idx[i]-1])+' '+str(verb[i]), str(doc[verb_idx[i]-1])+' '+vb_conj[i], answer2)
  
  #We replace first person pronouns by second person pronouns
  answer2=re.sub(r'\bi\b', 'you', answer2)
  answer2=re.sub(r'\bI\b', 'you', answer2)
  answer2=re.sub(r'\bmy\b', 'your', answer2)
  answer2=re.sub(r'\bMy\b', 'your', answer2)
  answer2=re.sub(r'\bmine\b', 'yours', answer2)
  answer2=re.sub(r'\bMine\b', 'yours', answer2)
  answer2=re.sub(r'\bme\b', 'you', answer2)

  answer2_copy=re.sub(r'\bi\b', 'you', answer2_copy)
  answer2_copy=re.sub(r'\bI\b', 'you', answer2_copy)
  answer2_copy=re.sub(r'\bmy\b', 'your', answer2_copy)
  answer2_copy=re.sub(r'\bMy\b', 'your', answer2_copy)
  answer2_copy=re.sub(r'\bmine\b', 'yours', answer2_copy)
  answer2_copy=re.sub(r'\bMine\b', 'yours', answer2_copy)
  answer2_copy=re.sub(r'\bme\b', 'you', answer2_copy)
  answer2_copy=re.sub(r'\bam\b', 'are', answer2_copy)
  answer2_copy=re.sub(r'\bwas\b', 'were', answer2_copy)
 

  ###### Stage 2: Asking about the experience ######
  print("- Socrate: Is it the first time "+answer2+"? (yes/no)")

  answer3="init"
  while answer3.lower()!="yes" and answer3.lower()!="no":
    answer3=input("- User: ")
    if answer3.lower()!="yes" and answer3.lower()!="no":
      print(wrapper.fill("- Socrate: Could you please respond with yes or no: Is it the first time "+answer2+"? (yes/no)"))
  
  ###### Stage 2.1 : First case: it is not the first time the problem happens #####
  if answer3.lower()=="no":
    
    ###### Stage 3: Making sure the statement is clear ######
    print(wrapper.fill("- Socrate: So "+answer1+" because "+answer2_copy+" and it is not the first it happens to you, is that correct? (yes/no)"))
    
    answer4="init"
    while answer4.lower()!="yes" and answer4.lower()!="no":
      answer4=input("- User: ")
      if answer4.lower()!="yes" and answer4.lower()!="no":
        print(wrapper.fill("- Socrate: Could you please respond with yes or no: I am correct? (yes/no)"))
  
    ##### Stage 3.1 : The statement is clear #####
    if answer4.lower()=="yes":

      ###### Stage 4: Conclusion of the conversation ######
      print(wrapper.fill("- Socrate: Well for some people, it takes more time to learn from a bad situation. But I believe that, this time, you will learn from it and you will not end up in this bad situation again"))
    
    ##### Stage 3.2 : The statement is not clear: ask again for the problem and the reasons #####
    if answer4.lower()=="no":
      print(wrapper.fill("- Socrate: I am sorry, there is something I did not get, can you state again what is your problem and what you believe the reasons of this problem are?"))
      answer3=input("- User: ")

      ###### Stage 4: Conclusion of the conversation ######
      print(wrapper.fill("- Socrate: Right, I am glad if I have helped you clarify your thinking."))
      



  ##### Stage 2.1 : First case: it is the first time the problem happens #####
  if answer3.lower()=="yes":
    
    ###### Stage 3: Making sure the statement is clear ######
    print(wrapper.fill("- Socrate: So "+answer1+" because "+answer2_copy+" and it is the first it happens to you, is that correct? (yes/no)"))
    
    answer4="init"
    while answer4.lower()!="yes" and answer4.lower()!="no":
      answer4=input("- User: ")
      if answer4.lower()!="yes" and answer4.lower()!="no":
        print(wrapper.fill("- Socrate: Could you please respond with yes or no: I am correct? (yes/no)"))
  
    ##### Stage 3.1 : The statement is clear #####
    if answer4.lower()=="yes":

      ###### Stage 4: Conclusion of the conversation ######
      print(wrapper.fill("- Socrate : So to make you feel better, think about why this happened today and why it has never happened before. This will help you solving your bad situation."))
    
    ##### Stage 3.1 : The statement is not clear: ask again for the problem and the reasons #####
    if answer4.lower()=="no":
      print(wrapper.fill("- Socrate: I am sorry, there is something I did not get, can you state again what is your problem and what you believe the reasons of this problem are?"))
      answer3=input("- User: ")

      ###### Stage 4: Conclusion of the conversation ######
      print(wrapper.fill("- Socrate: Right, I am glad if I have helped you clarify your thinking."))
      

chatbot()

- Socrate: Hello I am Socrate!, I am here to help you, what is your problem at the moment?
- User: I lost my wallet
- Socrate: Could you think of any reason why you lost your wallet?
- User: I forgot it at my office
- Socrate: Is it the first time you forget it at your office? (yes/no)
- User: no
- Socrate: So you lost your wallet because you forgot it at your office and it is not the first it happens to you, is
that correct? (yes/no)
- User: no
- Socrate: I am sorry, there is something I did not get, can you state again what is your problem and what you believe
the reasons of this problem are?
- User: Actually, I've just remembered that my wallet is in my handbag that stayed at home
- Socrate: Right, I am glad if I have helped you clarify your thinking.
